Requirement: local python installation on a Windows machine, with access to a chromedriver with GUI

In [1]:
# !pip install beautifulsoup4


In [2]:
# !pip install selenium


In [3]:
# !pip install pywin32


In [4]:
import pandas as pd, time, requests, json
import urllib.parse
from bs4 import BeautifulSoup
import win32clipboard
import numpy as np


In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# make sure the path is correct for your chromedriver. can place next to it
# driver = webdriver.Chrome('./chromedriver.exe')
# driver = webdriver.Chrome('C:/Program Files/chromedriver/chromedriver.exe')
driver = webdriver.Edge("C:/Program Files/chromedriver/msedgedriver.exe")
# opens Chrome, maximize window
driver.maximize_window()


C:\Users\csala\AppData\Local\Temp\ipykernel_20576\1201828961.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('C:/Program Files/chromedriver/msedgedriver.exe')


# Stage 0
Get keywords, construct mining chains

In [6]:
df = pd.read_excel("MilestoneAlumni-2011-2021-asof20220624.xlsx")
df.head()


Contact Id Family Name First Name         Full Name  GDPR Consent?  \
0  0031v00002Y0wXnAAJ        Ábel     Mihály       Ábel Mihály          False   
1  0031v00002Y0zRsAAJ     Ábrahám       Máté      Ábrahám Máté          False   
2  0031v00002Z9t8bAAB     Adorján     Dániel    Adorján Dániel          False   
3  0030Y00001C2NNbQAN     Adorján     Eszter    Adorján Eszter           True   
4  0030Y00001C2NNcQAN     Adorján   Viktória  Adorján Viktória           True   

     Email (if GDPR accepted)  Year of Graduation  \
0                  No consent                2013   
1                  No consent                2015   
2                  No consent                2017   
3     babiadorjan49@gmail.com                2019   
4  viktoria.adorjan@gmail.com                2020   

                                   Final Destination  
0  The University of Warwick - United Kingdom - E...  
1  Durham University - United Kingdom - Modern Eu...  
2  Durham University - United Kingdom - General E...  
3  Durham University - United Kingdom - Modern La...  
4  Imperial College London - United Kingdom - Com...

In [7]:
df["uni"] = [
    i.split(" - ")[0].split("(")[0].replace("The", "").strip()
    for i in df["Final Destination"]
]


# Stage 1
Mine profile links

In [8]:
driver.get("https://www.linkedin.com/")
# sign in
actions = ActionChains(driver)


In [9]:
cards = {}


In [100]:
# for k in kw:
for i in list(df.T.iteritems())[:]:
    if i[1]["uni"] == "No data":
        k = i[1]["Full Name"] + " Budapest Milestone"
    else:
        k = i[1]["Full Name"] + " " + i[1]["uni"]
    url = (
        "https://www.linkedin.com/search/results/people/?keywords="
        + urllib.parse.quote(k)
    )
    driver.get(url)
    time.sleep(0.3 + np.random.uniform(5) / 10.0)  # szunet random ideig oldalak kozott
    actions.send_keys(Keys.PAGE_DOWN)
    actions.perform()
    time.sleep(0.3 + np.random.uniform(5) / 10.0)  # szunet random ideig scrollok kozott
    soup = BeautifulSoup(driver.page_source, "html.parser")
    iid = i[1]["Contact Id"]
    if iid not in cards:
        cards[iid] = top_card = soup.find(
            "li", {"class": "reusable-search__result-container"}
        )
    if cards[iid]:
        cardsl = [str(i) for i in list(set(cards[iid]))]
        print(len(cardsl), url)
        open("cards/" + iid + ".txt", "w").write(json.dumps(cardsl))
    else:
        cards[iid]: "not found"


3 https://www.linkedin.com/search/results/people/?keywords=%C3%81bel%20Mih%C3%A1ly%20University%20of%20Warwick
3 https://www.linkedin.com/search/results/people/?keywords=%C3%81brah%C3%A1m%20M%C3%A1t%C3%A9%20Durham%20University
3 https://www.linkedin.com/search/results/people/?keywords=Adorj%C3%A1n%20D%C3%A1niel%20Durham%20University
3 https://www.linkedin.com/search/results/people/?keywords=Ag%C3%B3cs%20Fruzsina%20University%20of%20Cambridge
3 https://www.linkedin.com/search/results/people/?keywords=Agud%20Angelika%20University%20of%20Nottingham
3 https://www.linkedin.com/search/results/people/?keywords=Alm%C3%A1si%20D%C3%A1niel%20King%27s%20College%20London
3 https://www.linkedin.com/search/results/people/?keywords=Anda%20Vikt%C3%B3ria%20UCL
3 https://www.linkedin.com/search/results/people/?keywords=And%C3%B3%20Angelika%20University%20of%20Oxford
3 https://www.linkedin.com/search/results/people/?keywords=Antal%20Bal%C3%A1zs%20University%20of%20Exeter
3 https://www.linkedin.com/search/

Go, do the link extractor in `3`, create `links_full.json`, then come back for laggards

In [14]:
links_full = json.load(open("links_full.json", "r"))
nodes = json.load(open("nodes.json", "r"))


In [15]:
len(links_full), len(nodes)


(700, 698)

In [18]:
cards = {}

In [21]:
for i in list(df.T.iteritems())[:]:
  iid=i[1]['Contact Id']
  if iid not in links_full:
    if iid not in cards:    
      k=i[1]['Full Name']+' Budapest'
      url='https://www.linkedin.com/search/results/people/?keywords='+urllib.parse.quote(k)
      driver.get(url)
      time.sleep(0.3+np.random.uniform(5)/10.0) #szunet random ideig oldalak kozott
      actions.send_keys(Keys.PAGE_DOWN)
      actions.perform()   
      time.sleep(0.3+np.random.uniform(5)/10.0) #szunet random ideig scrollok kozott
      soup=BeautifulSoup(driver.page_source, 'html.parser')
      cards[iid]=soup.find('li',{'class':'reusable-search__result-container'})
      if cards[iid]:
       cardsl=[str(i) for i in list(set(cards[iid]))]
       print(len(cardsl),url) 
       open('cards/'+iid+'.txt','w').write(json.dumps(cardsl))
      else:
        cards[iid]:'not found'

3 https://www.linkedin.com/search/results/people/?keywords=Andr%C3%A1s%20Gy%C3%B6ri%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Arany%20%C3%81d%C3%A1m%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Bakonyi%20M%C3%A1rton%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Bellus%20Bal%C3%A1zs%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=B%C3%A1lint%20Martin%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Bal%C3%B3%20S%C3%A1ra%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=B%C3%A1rdos%20Fanni%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Barta%20Zsombor%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=B%C3%A9k%C3%A9s%20Anna%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Bencsik%20Eszter%20Budapest
3 https://www.linkedin.com/search/results/people/?keywords=Bencze%20Denisza%20Budapest
3 h

In [35]:
open('second_round.json','w').write(json.dumps([i for i in cards if cards[i]]))

3410

In [36]:
open('first_round.json','w').write(json.dumps(\
  [{links_full[i][0].split('/in/')[1].split('?')[0]:i for i in links_full}[j] for j in nodes]\
  ))

15356

In [37]:
first_round=json.load(open('first_round.json','r'))
second_round=json.load(open('second_round.json','r'))

In [38]:
[i for i in first_round if i in second_round]

[]

In [39]:
len(first_round),len(second_round)

(698, 155)

Extract links but manual check second rounds if plausible